In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import math
import sklearn
import numpy as np
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
%config IPcompletor.greedy = True
# print("Python: {}".format(sys.version))
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression


Change your path according to your dataset's directory

In [27]:
import csv
path='drive/My Drive/ml_colab/'
train_path=path+'rec_pred_train.csv'
test_path=path+'rec_pred_test.csv'

trainData = pd.read_csv(train_path, index_col = 0,error_bad_lines=True)#385688
testData = pd.read_csv(test_path, index_col = 0,error_bad_lines=False)#9146
testData.dropna(how="all", inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data Cleaning

In [0]:
trainData.replace(["\\N", "nA", "Not provided", "NaN"], np.nan, inplace=True)
testData.replace(["\\N", "nA", "Not provided", "NaN"], np.nan, inplace=True)

In [0]:
numeric_columns = [
    "query_word_count",
    "query_char_count",
    "query_document_id",
    "year_published",
    "number_of_authors",
    "abstract_char_count",
    "abstract_word_count",
    "first_author_id",
    "num_pubs_by_first_author",
    "hour_request_received",
    "local_hour_of_request",
    "recommendation_algorithm_id_used",
    "clicks",
]
time_columns = [
    "request_received",
    "response_delivered",
    "local_time_of_request",
    "time_recs_recieved",
    "time_recs_displayed",
    "time_recs_viewed",
]
for col in numeric_columns:
    trainData[col] = pd.to_numeric(trainData[col], errors="coerce")
    testData[col] = pd.to_numeric(testData[col], errors="coerce")
for col in time_columns:
    trainData[col] = pd.to_datetime(trainData[col], format="%d/%m/%Y %H:%M")
    testData[col] = pd.to_datetime(testData[col], format="%d/%m/%Y %H:%M")

In [30]:
trainData.isnull().sum().sort_values()

set_clicked                              0
ctr                                      0
rec_processing_time                      0
response_delivered                       0
hour_request_received                    0
request_received                         0
application_type                         0
organization_id                          0
algorithm_class                          0
number_of_recs_in_set                    0
search_title                             0
search_keywords                          0
search_abstract                          0
clicks                                   0
query_char_count                       154
query_word_count                       154
query_identifier                       154
country_by_ip                         1584
query_detected_language               3597
recommendation_algorithm_id_used     10677
app_lang                             16697
item_type                            36223
timezone_by_ip                       76209
local_hour_

In [0]:
# Splitting the dataset by organisation_id (1,4,8 ~ Blog, Jabref, MyVolts **in no specific order)

o_id1_train = trainData[trainData['organization_id'] == 1]
o_id4_train = trainData[trainData['organization_id'] == 4]
o_id8_train = trainData[trainData['organization_id'] == 8]

o_id1_test = testData[testData['organization_id'] == 1]
o_id4_test = testData[testData['organization_id'] == 4]
o_id8_test = testData[testData['organization_id'] == 8]


In [32]:
# Checking if the lengths match

print(len(trainData))
print(len(o_id1_train))
print(len(o_id4_train))
print(len(o_id8_train))
print('\n')
print(len(testData))
print(len(o_id1_test))
print(len(o_id4_test))
print(len(o_id8_test))

385687
270246
100215
15226


9145
5176
3029
940


In [33]:
### Printing the nan values by column for train data, classified by organisation_id

print('\n Organization Id 1')
print(o_id1_train.isnull().sum().sort_values())
print(o_id1_train.isnull().values.any()) #Prints True if atleast 1 Nan present
print(o_id1_train.isnull().sum().sum()) #Prints total no of nan values
print(len(o_id1_train)) #Prints total no of rows (length) in DataFrame

print('\n Organization Id 4')
print(o_id4_train.isnull().sum().sort_values())
print(o_id4_train.isnull().values.any())
print(o_id4_train.isnull().sum().sum())
print(len(o_id4_train))

print('\n Organization Id 4')
print(o_id8_train.isnull().sum().sort_values())
print(o_id8_train.isnull().values.any())
print(o_id8_train.isnull().sum().sum())
print(len(o_id8_train))
print('\n')


 Organization Id 1
set_clicked                              0
request_received                         0
hour_request_received                    0
response_delivered                       0
rec_processing_time                      0
ctr                                      0
number_of_recs_in_set                    0
organization_id                          0
algorithm_class                          0
search_keywords                          0
search_abstract                          0
query_char_count                         0
query_word_count                         0
query_identifier                         0
clicks                                   0
search_title                             0
application_type                         0
app_version                             84
item_type                              101
app_lang                              1154
country_by_ip                         1280
query_detected_language               3443
recommendation_algorithm_id_used  

In [34]:
### Printing the nan values by column for test data, classified by organisation_id
print('\n Organization Id 1')
print(o_id1_test.isnull().sum().sort_values())
print(o_id1_test.isnull().values.any())
print(o_id1_test.isnull().sum().sum())
print(len(o_id1_test))
print('\n Organization Id 4')
print(o_id4_test.isnull().sum().sort_values())
print(o_id4_test.isnull().values.any())
print(o_id4_test.isnull().sum().sum())
print(len(o_id4_test))
print('\n Organization Id 8')
print(o_id8_test.isnull().sum().sort_values())
print(o_id8_test.isnull().values.any())
print(o_id8_test.isnull().sum().sum())
print(len(o_id8_test))
print('\n')


 Organization Id 1
search_title                           0
query_identifier                       0
query_word_count                       0
query_char_count                       0
search_keywords                        0
search_abstract                        0
hour_request_received                  0
request_received                       0
algorithm_class                        0
application_type                       0
organization_id                        0
app_version                            2
item_type                              2
app_lang                              19
country_by_ip                         31
query_detected_language               51
recommendation_algorithm_id_used     229
timezone_by_ip                       329
local_hour_of_request                509
local_time_of_request                509
cbf_parser                          1379
query_document_id                   3645
year_published                      3737
number_of_authors                   3

In [35]:
### Printing the nan values by column for train data with set clicked 1, classified by organisation_id

set_1_train_od1=o_id1_train[o_id1_train['set_clicked'] == 1]
set_1_train_od4=o_id4_train[o_id4_train['set_clicked'] == 1]
set_1_train_od8=o_id8_train[o_id8_train['set_clicked'] == 1]

print('\n Organization Id 1 for set_clicked=1')
print(set_1_train_od1.isnull().sum().sort_values())
print(set_1_train_od1.isnull().values.any())
print(set_1_train_od1.isnull().sum().sum())
print(len(set_1_train_od1))
print('\n Organization Id 4 for set_clicked=1')

print(set_1_train_od4.isnull().sum().sort_values())
print(set_1_train_od4.isnull().values.any())
print(set_1_train_od4.isnull().sum().sum())
print(len(set_1_train_od4))
print('\n Organization Id 8 for set_clicked=1')

print(set_1_train_od8.isnull().sum().sort_values())
print(set_1_train_od8.isnull().values.any())
print(set_1_train_od8.isnull().sum().sum())
print(len(set_1_train_od8))
print('\n')


 Organization Id 1 for set_clicked=1
set_clicked                            0
request_received                       0
hour_request_received                  0
response_delivered                     0
rec_processing_time                    0
ctr                                    0
number_of_recs_in_set                  0
organization_id                        0
algorithm_class                        0
search_keywords                        0
search_abstract                        0
query_char_count                       0
query_word_count                       0
query_identifier                       0
clicks                                 0
search_title                           0
application_type                       0
app_version                            8
item_type                              8
query_detected_language               30
app_lang                              38
country_by_ip                         42
recommendation_algorithm_id_used     278
timezone_by_ip     

In [0]:
# Dropping columns that are completely and almost completely nan for set clicked 1 for each organisation
o_id1_train=o_id1_train.drop(["user_java_version", "user_os_version", "session_id","user_id","time_recs_viewed","time_recs_displayed","time_recs_recieved","user_timezone","user_os","document_language_provided","abstract_detected_language","abstract_char_count","abstract_word_count"], axis=1)
o_id1_test=o_id1_test.drop(["user_java_version", "user_os_version", "session_id","user_id","time_recs_viewed","time_recs_displayed","time_recs_recieved","user_timezone","user_os","document_language_provided","abstract_detected_language","abstract_char_count","abstract_word_count"], axis=1)

o_id4_train=o_id4_train.drop(["app_version","number_of_authors","user_timezone","user_os","num_pubs_by_first_author","first_author_id","user_os_version","user_java_version","year_published","document_language_provided","time_recs_viewed"],axis=1)
o_id4_test=o_id4_test.drop(["app_version","number_of_authors","user_timezone","user_os","num_pubs_by_first_author","first_author_id","user_os_version","user_java_version","year_published","document_language_provided","time_recs_viewed"],axis=1)

o_id8_train=o_id8_train.drop(["app_version","user_timezone","session_id","document_language_provided","year_published","number_of_authors","first_author_id","num_pubs_by_first_author","app_lang","user_os","user_os_version","user_java_version","user_id","time_recs_viewed"],axis=1)
o_id8_test=o_id8_test.drop(["app_version","user_timezone","session_id","document_language_provided","year_published","number_of_authors","first_author_id","num_pubs_by_first_author","app_lang","user_os","user_os_version","user_java_version","user_id","time_recs_viewed"],axis=1)

In [37]:
### Printing the nan values by column for train data, classified by organisation_id

print('\n Organization Id 1')
print(o_id1_train.isnull().sum().sort_values())
print(o_id1_train.isnull().values.any())
print(o_id1_train.isnull().sum().sum())
print(len(o_id1_train))
print('\n Organization Id 4')
print(o_id4_train.isnull().sum().sort_values())
print(o_id4_train.isnull().values.any())
print(o_id4_train.isnull().sum().sum())
print(len(o_id4_train))
print('\n Organization Id 8')
print(o_id8_train.isnull().sum().sort_values())
print(o_id8_train.isnull().values.any())
print(o_id8_train.isnull().sum().sum())
print(len(o_id8_train))
print('\n')


 Organization Id 1
query_identifier                         0
clicks                                   0
search_abstract                          0
search_keywords                          0
search_title                             0
algorithm_class                          0
number_of_recs_in_set                    0
ctr                                      0
response_delivered                       0
hour_request_received                    0
request_received                         0
rec_processing_time                      0
application_type                         0
query_word_count                         0
query_char_count                         0
set_clicked                              0
organization_id                          0
app_version                             84
item_type                              101
app_lang                              1154
country_by_ip                         1280
query_detected_language               3443
recommendation_algorithm_id_used  

In [0]:
# NOT THE BEST WAY TO FILL THE REMAINING NANs..... HAVE TO FIND ALTERNTATIVES FOR BETTER PERFORMANCE

o_id1_train[o_id1_train.columns]=o_id1_train[o_id1_train.columns].fillna(o_id1_train.mode(dropna=True).iloc[0])
o_id4_train[o_id4_train.columns]=o_id4_train[o_id4_train.columns].fillna(o_id4_train.mode(dropna=True).iloc[0])
o_id8_train[o_id8_train.columns]=o_id8_train[o_id8_train.columns].fillna(o_id8_train.mode(dropna=True).iloc[0])

o_id1_test[o_id1_test.columns]=o_id1_test[o_id1_test.columns].fillna(o_id1_train.mode(dropna=True).iloc[0])
o_id4_test[o_id4_test.columns]=o_id4_test[o_id4_test.columns].fillna(o_id4_train.mode(dropna=True).iloc[0])
o_id8_test[o_id8_test.columns]=o_id8_test[o_id8_test.columns].fillna(o_id8_train.mode(dropna=True).iloc[0])

In [39]:
### Printing the nan values by column for train data, classified by organisation_id
print('\n Organization Id 1')
print(o_id1_train.isnull().sum().sort_values())
print(o_id1_train.isnull().values.any())
print(o_id1_train.isnull().sum().sum())
print(len(o_id1_train))
print('\n Organization Id 4')
print(o_id4_train.isnull().sum().sort_values())
print(o_id4_train.isnull().values.any())
print(o_id4_train.isnull().sum().sum())
print(len(o_id4_train))
print('\n Organization Id 8')
print(o_id8_train.isnull().sum().sort_values())
print(o_id8_train.isnull().values.any())
print(o_id8_train.isnull().sum().sum())
print(len(o_id8_train))
print('\n')


 Organization Id 1
query_identifier                    0
clicks                              0
search_abstract                     0
search_keywords                     0
search_title                        0
cbf_parser                          0
algorithm_class                     0
recommendation_algorithm_id_used    0
number_of_recs_in_set               0
local_hour_of_request               0
local_time_of_request               0
timezone_by_ip                      0
country_by_ip                       0
app_lang                            0
app_version                         0
rec_processing_time                 0
response_delivered                  0
hour_request_received               0
request_received                    0
item_type                           0
application_type                    0
organization_id                     0
num_pubs_by_first_author            0
first_author_id                     0
number_of_authors                   0
year_published                

In [0]:
# Trying to make 3 classifiers, with o_id1_train, o_id4_train, o_id8_train